# Opportunity Watchlist

This notebook generates a practical SEO optimization queue from `seo_page_daily`.

Default logic focuses on pages with:
- high impressions
- low CTR
- mid rankings (positions where small improvements can produce clicks)

In [ ]:
import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../..")

import plotly.express as px

import lifeline_theme
from lla_data import config
from lla_data.bq import build_date_params, default_query_window, get_client, run_query

lifeline_theme.inject_fonts()

client = get_client()
window = default_query_window(config.DEFAULT_DAYS_BACK)

In [ ]:
query = f"""
SELECT
  page_path,
  SUM(gsc_clicks) AS clicks,
  SUM(gsc_impressions) AS impressions,
  SAFE_DIVIDE(SUM(gsc_clicks), NULLIF(SUM(gsc_impressions), 0)) AS ctr,
  SAFE_DIVIDE(SUM(gsc_avg_position * gsc_impressions), NULLIF(SUM(gsc_impressions), 0)) AS avg_position,
  SUM(organic_sessions) AS organic_sessions
FROM `{config.PROJECT_ID}.{config.SEARCHCONSOLE_DATASET}.seo_page_daily`
WHERE report_date BETWEEN DATE(@start_date) AND DATE(@end_date)
GROUP BY page_path
HAVING impressions >= 100
ORDER BY impressions DESC
"""

df_watch = run_query(client, query, params=build_date_params(window))

df_watchlist = df_watch[
    (df_watch["ctr"] < 0.05)
    & (df_watch["avg_position"] >= 6)
    & (df_watch["avg_position"] <= 20)
].sort_values(["impressions", "ctr"], ascending=[False, True])

df_watchlist.head(30)

In [ ]:
fig = px.scatter(
    df_watchlist,
    x="avg_position",
    y="ctr",
    size="impressions",
    hover_name="page_path",
    template="lifeline",
    title="Opportunity Watchlist: Low CTR + Mid Position + High Impressions",
)
fig.update_yaxes(tickformat=".0%")
lifeline_theme.add_lifeline_logo(fig)
fig.show()